In [1]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# You'll generate plots of attention in order to see which p arts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np  
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
import cv2
from sklearn.utils import shuffle

from IPython.display import clear_output, display
import os.path
from os import path
import shutil
import PIL
from pathlib import Path
from matplotlib import pyplot as plt

from IPython.display import clear_output

import numpy as np

from PIL import ImageFont
from PIL import Image, ImageDraw
from PIL import ImageColor

from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

import random

import time
import shutil
import uuid
from shutil import copyfile
from collections import Counter

In [2]:
def generate_dataset_v2_8lines( name,
                               n_hand, 
                               n_parts_rand, 
                               n_parts_ec,
                               n_parts_seq,
                               n_torneio):
    # cria diretorio
    root= 'C:/mestrado/repos-github/chess-attention/dataset-builder/datasets-8lines/'+ name
    Path( os.path.join( root, 'train/images')).mkdir(parents=True, exist_ok=True)
    Path( os.path.join( root, 'train/labels')).mkdir(parents=True, exist_ok=True)
    Path( os.path.join( root, 'valid/images')).mkdir(parents=True, exist_ok=True)
    Path( os.path.join( root, 'valid/labels')).mkdir(parents=True, exist_ok=True)
    
    parts = []
    parts.append( (n_hand, 'hand')) if n_hand!= 0 else None
    parts.append( (n_parts_rand, 'parts-random')) if n_parts_rand!= 0 else None
    parts.append( (n_parts_seq, 'parts-seq')) if n_parts_seq!= 0 else None
    parts.append( (n_parts_ec, 'parts-ec')) if n_parts_ec!= 0 else None
    parts.append( (n_torneio, 'torneio-except-test')) if n_torneio!= 0 else None
    
    files= []
    pool_dir = 'C:/mestrado/repos-github/chess-attention/dataset-builder/pool_8lines/'
    for (n, folder) in parts:
        print( folder, n)
        parts_files = glob( os.path.join( pool_dir, folder, 'labels/*.pgn'))
        files.extend( parts_files[:n] if n> 0 else parts_files)
    
#     N_train= int( len(files)*.8)
    N_train= 10000  #aqui usa fixo
    print( 'train, valid', N_train, len(files)-N_train)
    
    # shuffle
    random.shuffle( files)
    print( 'total', len( files))
    
    # copia
    for i in range( 0, len( files)):
        if( i%1000== 0):
            print( i, '....')
        
        f= files[i]
        folder= 'train' if i< N_train else 'valid'
        
        # copia
        dest_file= os.path.join( root, folder, 'labels', Path(f).name)
#         print( dest_file)
        shutil.copyfile( f, dest_file)
        shutil.copyfile( 
            f.replace( 'labels', 'images').replace( 'pgn', 'jpg'),                  
            dest_file.replace( 'labels', 'images').replace( 'pgn', 'jpg'))

    

In [9]:
generate_dataset_v2_8lines( name='-8linhas-curriculum--unico--5k',
                           n_hand=417, 
                           n_torneio= 78,
                           n_parts_rand=2750, 
                           n_parts_seq= 1485,
                           n_parts_ec= 770,
                           )

hand 417
parts-random 2750
parts-seq 1485
parts-ec 770
torneio-except-test 78
train, valid 5000 500
total 5500
0 ....
1000 ....
2000 ....
3000 ....
4000 ....
5000 ....


In [10]:
generate_dataset_v2_8lines( name='-8linhas-handwritten--5k',
                           n_hand=2010, 
                           n_torneio= 378,
                           n_parts_rand=0, 
                           n_parts_seq= 3112,
                           n_parts_ec= 0,
                           )

hand 2010
parts-seq 3112
torneio-except-test 378
train, valid 5000 500
total 5500
0 ....
1000 ....
2000 ....
3000 ....
4000 ....
5000 ....


In [3]:
generate_dataset_v2_8lines( name='-8linhas-handwritten-only-2388',
                           n_hand=2010, 
                           n_torneio= 378,
                           n_parts_rand=0, 
                           n_parts_seq= 0,
                           n_parts_ec= 0,
                           )

hand 2010
torneio-except-test 378
train, valid 2000 388
total 2388
0 ....
1000 ....
2000 ....


In [4]:
generate_dataset_v2_8lines( name='-8linhas-random-only-2388',
                           n_hand=0, 
                           n_torneio= 0,
                           n_parts_rand=2388, 
                           n_parts_seq= 0,
                           n_parts_ec= 0,
                           )

parts-random 2388
train, valid 2000 388
total 2388
0 ....
1000 ....
2000 ....


In [5]:
generate_dataset_v2_8lines( name='curriculum-8-linhas--etapa-6-5500',
                           n_hand=2010, 
                           n_torneio= 378,
                           n_parts_rand=1012, 
                           n_parts_seq= 1100,
                           n_parts_ec= 1000,
                           )

hand 2010
parts-random 1012
parts-seq 1100
parts-ec 1000
torneio-except-test 378
train, valid 2000 3500
total 5500
0 ....
1000 ....
2000 ....
3000 ....
4000 ....
5000 ....


In [3]:
generate_dataset_v2_8lines( name='-8linhas-handwritten--10k',
                           n_hand=2010, 
                           n_torneio= 378,
                           n_parts_rand=0, 
                           n_parts_seq= 3112+5500,
                           n_parts_ec= 0,
                           )

hand 2010
parts-seq 8612
torneio-except-test 378
train, valid 10000 1000
total 11000
0 ....
1000 ....
2000 ....
3000 ....
4000 ....
5000 ....
6000 ....
7000 ....
8000 ....
9000 ....
10000 ....
